# Decision Tree with CART using Information gain, Gini Impurity.

In [1]:
# Toy dataset.
# Format: each row is an example.
# The last column is the label.
# The first two columns are features.
# Feel free to play with it by adding more features & examples.
# Interesting note: I've written this so the 2nd and 5th examples
# have the same features, but different labels - so we can see how the
# tree handles this case.
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [2]:
# Column labels.
# These are used only to print the tree.
header = ["color", "diameter", "label"]

In [3]:
print(header)
training_data

['color', 'diameter', 'label']


[['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [4]:
def unique_vals(rows, col):
    """Find the unique values for a column in a given dataset."""
    return set([row_element[col] for row_element in rows])

In [5]:
# get the unique entires in the required cols
unique_vals(training_data,0)

{'Green', 'Red', 'Yellow'}

In [58]:
def class_counts(rows):
    """ Counts the number of observations that belong to the final target class. 
        This function assumes the target label is the last column always. """
    
    # class : observation counts
    counts = {}
    
    # for every row check if the element is present & update accordingly.
    for row_element in rows:
        if row_element[-1] not in counts:
            counts[row_element[-1]] = 0
        counts[row_element[-1]]+=1
    return counts

In [59]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [26]:
def is_numeric(value):
    """checks if an entry is a number."""
    return isinstance(value, int) or isinstance(value, float)

In [27]:
class Question:
    """A Question is used to partition the dataset.

    This class supports to ask questions on the dataset.
    match() checks if the dataset specified  meets the required condition.
    The condition is usually to check if the datapoint > number in the case of a numerical,
    or equals in case of a string.
    repr() makes the question more understandable - readable.
    """
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, example):
        """
        example: Sample datapoint to check on.
        """
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        return val == self.value
    
    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = '=='
        
        if is_numeric(self.value):
            condition = '>='
        
        return "Is %s %s %s?" % (
            header[self.column],condition,str(self.value))
            

In [28]:
q = Question(0,'Red')

In [29]:
Question(0,'Yellow').match(training_data[0])

False

In [30]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows = []
    false_rows = []
    
    for row_element in rows:
        if question.match(row_element):
            true_rows.append(row_element)
        else:
            false_rows.append(row_element)
    
    return true_rows,false_rows

In [31]:
true_rows, false_rows = partition(training_data, Question(0, 'Red'))

In [32]:
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [33]:
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

### Gini impurity at a node is the is the chance a randomly selected data point and a randomly selected target label in the dataset are incorrect match - misclassification.

In [60]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)    
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [64]:
# what is the sample gini impurity 
gini([['Apple'],
      ['Apple'],
      ['fruit'],
      ['Yellow']])

0.625

In [65]:
gini(training_data)

0.6399999999999999

In [66]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - (p*gini(left)+ (1-p)*gini(right))

### How much information do we gain by partioning on 'Green'?


In [68]:
current_uncertainty = gini(training_data)

In [69]:
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

### How much information do we gain by partioning on 'Red'?


In [70]:
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [71]:
print(true_rows)
false_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]


[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

### Finding best split - such that we ask the right question is the challenge, as we can see it must be automated to ask the right questions and make a splitting decision accordingly.